<a href="https://colab.research.google.com/github/Alex-Steel-13/datascience/blob/main/Handwritten_numbers.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
###DO NOT EDIT THIS CODE
################################################################################################################################

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torchvision import datasets, transforms
from PIL import Image
import matplotlib.pyplot as plt

# GPUs are 3x faster than CPU. Better to use if it is available 
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# Define Loss Function
loss_function = nn.CrossEntropyLoss()

# This function returns the number of parameters in the model
def num_params(model):
  return sum([p.numel() for p in model.parameters()])

# Define a Training Function. This function will: compute the forward pass, backpropagate,
# update the weights, and repeat the steps for a given number of epochs. At each epoch, 
# it will output the training loss and test loss at every step
def train(epochs, model, trainloader, testloader, optimizer, loss_function):
  for epoch in range(epochs):
    loss_epoch = np.array([])
    train_correct, train_total = 0, 0
    test_correct, test_total = 0, 0

    for data, labels in trainloader:
      # convert into GPU objects if needed
      input_data = data.to(device)
      labels = labels.to(device)

      # forward pass
      predict = model(input_data)
      
      # backward pass
      loss = loss_function(predict, labels)
      optimizer.zero_grad()
      loss.backward()

      # update parameters (weights and biases)
      optimizer.step()

      # store progress
      loss_epoch = np.append(loss_epoch, loss.item())

    # evaluate test accuracy
    for data, labels in testloader:
      input_data = data.to(device)
      labels = labels.to(device)
      predict = model(input_data)
      for i, out in enumerate(predict):
        pred = torch.argmax(out)
        if pred == labels[i]:
          test_correct+=1
        test_total+=1

    test_accuracy = test_correct/test_total    
  
    print('epoch [{}/{}], training loss:{:.4f}, test accuracy:{:.4f}'.format(epoch+1, epochs, np.mean(loss_epoch), test_accuracy))
################################################################################################################################

# **Load Dataset**

Available datasets are: MNIST, CIFAR10

## MNIST

In [2]:
# download and load data
batch_size = 512

# download and transform train dataset
train_loader = torch.utils.data.DataLoader(datasets.MNIST('./mnist_data', download=True, train=True, transform=transforms.Compose([
                                                transforms.ToTensor(), # first, convert image to PyTorch tensor
                                                transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
                                                ])), batch_size=batch_size, shuffle=True)

# download and transform test dataset
test_loader = torch.utils.data.DataLoader(datasets.MNIST('./mnist_data', download=True, train=False, transform=transforms.Compose([
                                                              transforms.ToTensor(), # first, convert image to PyTorch tensor
                                                              transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
                                                          ])), batch_size=batch_size, shuffle=True)

  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/train-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/train-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./mnist_data/MNIST/raw



  0%|          | 0/4542 [00:00<?, ?it/s]

Extracting ./mnist_data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./mnist_data/MNIST/raw



In [3]:
# it is a good idea to take a look at the data. Here we see it is a 28x28 grayscale image
for data, labels in train_loader:
  print(data[0].size())
  break

torch.Size([1, 28, 28])


# **Build a Network and Define Hyperparameters**

In [6]:
########################
####                #### 
#### EDIT THIS CELL ####
####                ####
########################


learning_rate = 10e-3
weight_decay = 10e-5
n_epochs = 10

# neural network
class NeuralNetwork(nn.Module):
  def __init__(self):
    super(NeuralNetwork, self).__init__()
    ### Define Layers
    self.ll1 = nn.Linear(784,128)
    self.ll2 = nn.Linear(128,64)
    self.ll3 = nn.Linear(64,10)

  def forward(self, x):
    x = torch.flatten(x,1)
    x = self.ll1(x)
    x = torch.sigmoid(x)
    x = self.ll2(x)
    x = torch.sigmoid(x)
    x = self.ll3(x)
    x = torch.sigmoid(x)
    return x
  
# Every time you edit the neural network, you'll have to update this cell
# Create model object
model = NeuralNetwork().to(device)

# Loads Adam optimizer, which implements a version of gradient descent
optimizer = optim.Adam(model.parameters(), lr=learning_rate, weight_decay=weight_decay)

In [7]:
# check the structure of your network
print(model)

# apply your model to a single input. This helps check that 
# the dimensions are correct
model(torch.rand(1,1,28,28, device=device))

NeuralNetwork(
  (ll1): Linear(in_features=784, out_features=128, bias=True)
  (ll2): Linear(in_features=128, out_features=64, bias=True)
  (ll3): Linear(in_features=64, out_features=10, bias=True)
)


tensor([[0.5777, 0.4160, 0.4878, 0.5241, 0.5987, 0.5305, 0.6291, 0.4463, 0.4897,
         0.5057]], device='cuda:0', grad_fn=<SigmoidBackward0>)

# **Train and Validate**

In [8]:
train(n_epochs, model, train_loader, test_loader, optimizer, loss_function)

epoch [1/10], training loss:1.6904, test accuracy:0.8504
epoch [2/10], training loss:1.5413, test accuracy:0.9232
epoch [3/10], training loss:1.5191, test accuracy:0.9557
epoch [4/10], training loss:1.5096, test accuracy:0.9547
epoch [5/10], training loss:1.5060, test accuracy:0.9624
epoch [6/10], training loss:1.5052, test accuracy:0.9573
epoch [7/10], training loss:1.5035, test accuracy:0.9603
epoch [8/10], training loss:1.5029, test accuracy:0.9661
epoch [9/10], training loss:1.5018, test accuracy:0.9634
epoch [10/10], training loss:1.5019, test accuracy:0.9622
